<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

<img width="20%" alt="Naas" src="https://logos-world.net/wp-content/uploads/2020/04/Linkedin-Logo.png">

# LinkedIn - Get posts likes
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/LinkedIn_get_posts_likes.ipynb" target="_parent">
<img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/>
</a>

## Input

In [ ]:
# Credentials
LI_AT = ""
JSESSIONID = ""

# Linkedin Post
POST = ""

## Model

### Imports

In [ ]:
import pandas as pd
import naas_drivers
import requests
import urllib

### Set up LinkedIn Session

In [ ]:
session = naas_drivers.linkedin.connect(LI_AT, JSESSIONID)

# Init cookies
cookies = session.cookies

# Init headers
headers = session.headers

### Functions to get post likes

In [ ]:
def get_post_urn(post_link):
    response = requests.get(post_link).text
    urn_index = response.index('urn:li:activity:')
    finish_index = response.index('"', urn_index)
    activity_urn = response[urn_index:finish_index]
    activity_urn = activity_urn.rsplit("?")[0]
    return activity_urn

def get_post_likes(post_link=None, thread_urn=None, count=100, start=0):
    if post_link:
        thread_urn = urllib.parse.quote(get_post_urn(post_link), safe='')
        
    if not thread_urn:
        print("Error, specify a 'post_link' or a 'thread_urn'")
        return None
    
    user = {'URN_ID': [],
            'PUBLIC_IDENTIFIER': [],
            'FIRSTNAME': [],
            'LASTNAME': [],
            'JOB_TITLE': [],
            }
    
    reacts = {'URN_ID': [],
              'REACTION_TYPE': []
              }
    while True:
        try:
            res = requests.get(f"https://www.linkedin.com/voyager/api/feed/reactions?count={count}&q=reactionType&start={start}&threadUrn={thread_urn}",
                               cookies=cookies,
                               headers=headers).json()
            for elem in res.get('included'):
                if elem.get('$type') =='com.linkedin.voyager.identity.shared.MiniProfile':
                    user['URN_ID'].append(elem.get('entityUrn').replace("urn:li:fs_miniProfile:", ""))
                    user['PUBLIC_IDENTIFIER'].append(elem.get('publicIdentifier'))
                    user['FIRSTNAME'].append(elem.get('firstName'))
                    user['LASTNAME'].append(elem.get('lastName'))
                    user['JOB_TITLE'].append(elem.get('occupation'))
                if elem.get("$type") == 'com.linkedin.voyager.feed.social.Reaction':
                    reacts['URN_ID'].append(elem.get('actorUrn').replace("urn:li:fs_miniProfile:", ""))
                    reacts['REACTION_TYPE'].append(elem['reactionType'])
            if "paging" in res.get('data'):
                start += count
                if (res.get('data').get('paging').get('total') < start):
                    break
        except:
            break
    df_user = pd.DataFrame(user)
    df_reacts = pd.DataFrame(reacts)
    
    df = pd.merge(df_user, df_reacts, on="URN_ID", how="left")
    df["POST_URL"] = post_link
    return df

## Output

In [ ]:
post_reactions = get_post_reactions(post_link=POST)
post_reactions